In [34]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
from sqlalchemy import Table, MetaData, Column, Integer, create_engine
import urllib
import pyodbc 
import math
import datetime

## Build and Establish Connection to DB

In [58]:
#server = ''
#database = ''
#username = ''
#password = ''


params = urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

engine = sa.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params, echo=True)

In [59]:
connection = engine.connect()

2022-05-25 17:56:54,659 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2022-05-25 17:56:54,661 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-25 17:56:54,712 INFO sqlalchemy.engine.Engine SELECT schema_name()
2022-05-25 17:56:54,715 INFO sqlalchemy.engine.Engine [generated in 0.00312s] ()
2022-05-25 17:56:54,798 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2022-05-25 17:56:54,799 INFO sqlalchemy.engine.Engine [generated in 0.00091s] ()


## Load Raw Eviction Notices Data

In [4]:
get_evictions = """
SELECT *
FROM SF_Eviction_Notices_Raw;
"""

evictions = pd.read_sql_query(get_evictions, connection)
evictions

2022-05-24 21:30:39,241 INFO sqlalchemy.engine.Engine 
SELECT *
FROM SF_Eviction_Notices_Raw;

2022-05-24 21:30:39,243 INFO sqlalchemy.engine.Engine [raw sql] ()


,id,eviction_id,address,city,state,zip,file_date,non_payment,breach,nuisance,...,other_cause,late_payments,lead_remediation,development,good_samaritan_ends,supervisor_district,neighborhood,shape,constraints_date,client_location
0,0,M220284,2000 Block Of Broadway Street,San Francisco,CA,94123,2022-02-18T00:00:00.000,False,True,False,...,False,False,False,False,False,2,Pacific Heights,"{'type': 'Point', 'coordinates': [-122.430824,...",nan,nan
1,1,M220262,500 Block Of Beale Street,San Francisco,CA,94105,2022-02-14T00:00:00.000,False,True,False,...,False,False,False,False,False,6,Financial District/South Beach,"{'type': 'Point', 'coordinates': [-122.38909, ...",nan,nan
2,2,M220049,2000 Block Of Broadway Street,San Francisco,CA,94123,2022-01-07T00:00:00.000,False,True,False,...,False,False,False,False,False,2,Pacific Heights,"{'type': 'Point', 'coordinates': [-122.430824,...",nan,nan
3,3,M211708,100 Block Of Bartlett Street,San Francisco,CA,94115,2021-11-12T00:00:00.000,False,False,True,...,False,False,False,False,False,9,Mission,"{'type': 'Point', 'coordinates': [-122.41979, ...",nan,nan
4,4,M211587,700 Block Of Fillmore Street,San Francisco,CA,94134,2021-10-25T00:00:00.000,False,False,True,...,False,False,False,False,False,5,Hayes Valley,"{'type': 'Point', 'coordinates': [-122.431305,...",nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43388,43392,S000967,1200 Block Of 04th Avenue,San Francisco,CA,94107,1997-12-08T00:00:00.000,False,False,False,...,False,False,False,False,False,5,Inner Sunset,"{'type': 'Point', 'coordinates': [-122.46107, ...",nan,nan
43389,43388,E992050,900 Block Of Mission Street,San Francisco,CA,94103,1999-10-12T00:00:00.000,False,False,False,...,False,False,False,False,False,6,South of Market,"{'type': 'Point', 'coordinates': [-122.40798, ...",nan,"{'latitude': '37.78154530291944', 'longitude':..."
43390,43389,RE01899,500 Block Of Wisconsin Street,San Francisco,CA,94107,1997-05-21T00:00:00.000,False,False,False,...,False,False,False,False,False,10,Potrero Hill,"{'type': 'Point', 'coordinates': [-122.39924, ...",nan,"{'latitude': '37.76048824432999', 'longitude':..."
43391,43390,RE01981,200 Block Of Central Avenue,San Francisco,CA,94117,1997-06-03T00:00:00.000,False,False,False,...,False,False,False,False,False,5,Haight Ashbury,"{'type': 'Point', 'coordinates': [-122.44393, ...",nan,"{'latitude': '37.77182670111496', 'longitude':..."


## Pull Target Rel_Eviction_Notices Table Structure

In [5]:
evictions_table_target_names = """
select 
    col.column_id as id,
    col.name,
    t.name as data_type,
    col.max_length,
    col.precision,
    col.is_nullable
from sys.tables as tab
    inner join sys.columns as col
        on tab.object_id = col.object_id
    left join sys.types as t
    on col.user_type_id = t.user_type_id
where tab.name = 'Rel_Eviction_Notices'
order by tab.name, column_id;
"""

evictions_table_fields = pd.read_sql_query(evictions_table_target_names, connection)
evictions_table_fields

2022-05-24 21:31:21,700 INFO sqlalchemy.engine.Engine 
select 
    col.column_id as id,
    col.name,
    t.name as data_type,
    col.max_length,
    col.precision,
    col.is_nullable
from sys.tables as tab
    inner join sys.columns as col
        on tab.object_id = col.object_id
    left join sys.types as t
    on col.user_type_id = t.user_type_id
where tab.name = 'Rel_Eviction_Notices'
order by tab.name, column_id;

2022-05-24 21:31:21,702 INFO sqlalchemy.engine.Engine [raw sql] ()


,id,name,data_type,max_length,precision,is_nullable
0,1,eviction_auto_id,uniqueidentifier,16,0,False
1,2,eviction_id,varchar,-1,0,False
2,3,file_date,date,3,10,False
3,4,non_payment,bit,1,1,False
4,5,breach,bit,1,1,False
5,6,nuisance,bit,1,1,False
6,7,illegal_use,bit,1,1,False
7,8,failure_to_sign_renewal,bit,1,1,False
8,9,access_denial,bit,1,1,False
9,10,unapproved_subtenant,bit,1,1,False


In [6]:
#view raw table column names
evictions.columns

Index(['id', 'eviction_id', 'address', 'city', 'state', 'zip', 'file_date',
       'non_payment', 'breach', 'nuisance', 'illegal_use',
       'failure_to_sign_renewal', 'access_denial', 'unapproved_subtenant',
       'owner_move_in', 'demolition', 'capital_improvement',
       'substantial_rehab', 'ellis_act_withdrawal', 'condo_conversion',
       'roommate_same_unit', 'other_cause', 'late_payments',
       'lead_remediation', 'development', 'good_samaritan_ends',
       'supervisor_district', 'neighborhood', 'shape', 'constraints_date',
       'client_location'],
      dtype='object')

## Pull Required Columns

In [7]:
evictions_columns = ['eviction_id', 'file_date','non_payment', 'breach', 'nuisance', 'illegal_use',
       'failure_to_sign_renewal', 'access_denial', 'unapproved_subtenant',
       'owner_move_in', 'demolition', 'capital_improvement',
       'substantial_rehab', 'ellis_act_withdrawal', 'condo_conversion',
       'roommate_same_unit', 'other_cause', 'late_payments',
       'lead_remediation', 'development', 'good_samaritan_ends', 'constraints_date']
evictions_grouped = evictions.groupby(evictions_columns).count().reset_index()
evictions_grouped = evictions_grouped[evictions_columns]
evictions_grouped

,eviction_id,file_date,non_payment,breach,nuisance,illegal_use,failure_to_sign_renewal,access_denial,unapproved_subtenant,owner_move_in,...,substantial_rehab,ellis_act_withdrawal,condo_conversion,roommate_same_unit,other_cause,late_payments,lead_remediation,development,good_samaritan_ends,constraints_date
0,AL2K0014,2000-02-11T00:00:00.000,False,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,nan
1,E2K2588,2000-09-19T00:00:00.000,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,nan
2,E980002,1998-01-02T00:00:00.000,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,nan
3,E980003,1998-01-02T00:00:00.000,False,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,nan
4,E980004,1998-01-02T00:00:00.000,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43387,S001123,1997-12-31T00:00:00.000,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,nan
43388,S001124,1997-12-31T00:00:00.000,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,nan
43389,S001125,1997-12-31T00:00:00.000,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,nan
43390,T2K2589,2000-09-19T00:00:00.000,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,nan


## Check for Nulls or Anomalies

In [8]:
#looks like nothing is registering as null
evictions_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43392 entries, 0 to 43391
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   eviction_id              43392 non-null  object
 1   file_date                43392 non-null  object
 2   non_payment              43392 non-null  object
 3   breach                   43392 non-null  object
 4   nuisance                 43392 non-null  object
 5   illegal_use              43392 non-null  object
 6   failure_to_sign_renewal  43392 non-null  object
 7   access_denial            43392 non-null  object
 8   unapproved_subtenant     43392 non-null  object
 9   owner_move_in            43392 non-null  object
 10  demolition               43392 non-null  object
 11  capital_improvement      43392 non-null  object
 12  substantial_rehab        43392 non-null  object
 13  ellis_act_withdrawal     43392 non-null  object
 14  condo_conversion         43392 non-nul

### Eviction Ids

In [9]:
# identify unusual eviction ids by value length
# 3 eviction ids appear to have length other than 7 chars
evictions_grouped.eviction_id.apply(len).value_counts()

7     43389
8         2
24        1
Name: eviction_id, dtype: int64

In [10]:
# the 3 anomaly eviction id's are unusual, but the data appears valid otherwise
# because we will use a substitute UUID to identify the evictions anyway, we've decided not to drop these values
evictions_grouped[evictions_grouped.eviction_id.apply(len) > 7]

,eviction_id,file_date,non_payment,breach,nuisance,illegal_use,failure_to_sign_renewal,access_denial,unapproved_subtenant,owner_move_in,...,substantial_rehab,ellis_act_withdrawal,condo_conversion,roommate_same_unit,other_cause,late_payments,lead_remediation,development,good_samaritan_ends,constraints_date
0,AL2K0014,2000-02-11T00:00:00.000,False,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,nan
40902,R005-26E,1997-01-21T00:00:00.000,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,nan
43391,on at lease one occasion,2000-02-14T00:00:00.000,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,nan


In [11]:
evictions[evictions.eviction_id == "on at lease one occasion"]

,id,eviction_id,address,city,state,zip,file_date,non_payment,breach,nuisance,...,other_cause,late_payments,lead_remediation,development,good_samaritan_ends,supervisor_district,neighborhood,shape,constraints_date,client_location
36329,36302,on at lease one occasion,1500 Block Of 19th Avenue,San Francisco,CA,94122,2000-02-14T00:00:00.000,False,False,False,...,False,False,False,False,False,7,Inner Sunset,"{'type': 'Point', 'coordinates': [-122.47682, ...",nan,"{'latitude': '37.758898682240776', 'longitude'..."


In [12]:
evictions[evictions.eviction_id == "AL2K0014"]

,id,eviction_id,address,city,state,zip,file_date,non_payment,breach,nuisance,...,other_cause,late_payments,lead_remediation,development,good_samaritan_ends,supervisor_district,neighborhood,shape,constraints_date,client_location
41477,41466,AL2K0014,1600 Block Of Howard Street,San Francisco,CA,94103,2000-02-11T00:00:00.000,False,True,False,...,False,True,False,False,False,6,Mission,"{'type': 'Point', 'coordinates': [-122.417274,...",nan,"{'latitude': '37.771072990654986', 'longitude'..."


In [13]:
evictions[evictions.eviction_id == "R005-26E"]

,id,eviction_id,address,city,state,zip,file_date,non_payment,breach,nuisance,...,other_cause,late_payments,lead_remediation,development,good_samaritan_ends,supervisor_district,neighborhood,shape,constraints_date,client_location
36352,36327,R005-26E,3900 Block Of Sacramento Street,San Francisco,CA,nan,1997-01-21T00:00:00.000,False,False,False,...,False,False,False,False,False,2,Presidio Heights,"{'type': 'Point', 'coordinates': [-122.45807, ...",nan,"{'latitude': '37.78677576518167', 'longitude':..."


### File Dates

In [14]:
# identify unusual file dates by value length
# no apparent anomalies
evictions_grouped.file_date.apply(len).value_counts()

23    43392
Name: file_date, dtype: int64

### Binary Eviction Notice Values

In [15]:
#inspect breakdown of T/F in binary columns, confirms no NaNs
for i in range(2,21):
    print(evictions_grouped.iloc[:,i].value_counts())

False    40667
True      2725
Name: non_payment, dtype: int64
False    33667
True      9725
Name: breach, dtype: int64
False    35542
True      7850
Name: nuisance, dtype: int64
False    42476
True       916
Name: illegal_use, dtype: int64
False    43292
True       100
Name: failure_to_sign_renewal, dtype: int64
False    43063
True       329
Name: access_denial, dtype: int64
False    42584
True       808
Name: unapproved_subtenant, dtype: int64
False    33090
True     10302
Name: owner_move_in, dtype: int64
False    42215
True      1177
Name: demolition, dtype: int64
False    41688
True      1704
Name: capital_improvement, dtype: int64
False    43310
True        82
Name: substantial_rehab, dtype: int64
False    39262
True      4130
Name: ellis_act_withdrawal, dtype: int64
False    43263
True       129
Name: condo_conversion, dtype: int64
False    41998
True      1394
Name: roommate_same_unit, dtype: int64
False    41979
True      1413
Name: other_cause, dtype: int64
False    41729
True

### Constraint Dates

In [16]:
#39211 nan values
evictions_grouped.constraints_date.value_counts()

nan                        39211
2021-06-20T00:00:00.000        9
2005-08-01T00:00:00.000        9
2007-06-01T00:00:00.000        7
2018-05-01T00:00:00.000        7
                           ...  
2021-08-08T00:00:00.000        1
2013-03-26T00:00:00.000        1
2009-05-08T00:00:00.000        1
2011-10-16T00:00:00.000        1
2024-06-17T00:00:00.000        1
Name: constraints_date, Length: 2646, dtype: int64

In [17]:
#replace nan string with np.NaN values
evictions_grouped.constraints_date = evictions_grouped.constraints_date.replace('nan', np.NaN)

In [18]:
#check
evictions_grouped.constraints_date.isna().sum()

39211

## Cleaning
### Uppercase

In [19]:
#create new df
evictions_cleaned = evictions_grouped

In [20]:
#change all eviction IDs to uppercase
evictions_cleaned.eviction_id = evictions_grouped.eviction_id.str.upper()
evictions_cleaned.eviction_id

0                        AL2K0014
1                         E2K2588
2                         E980002
3                         E980003
4                         E980004
                   ...           
43387                     S001123
43388                     S001124
43389                     S001125
43390                     T2K2589
43391    ON AT LEASE ONE OCCASION
Name: eviction_id, Length: 43392, dtype: object

### Date Format

In [21]:
#convert file_date strings to datetime
evictions_cleaned.file_date = pd.to_datetime(evictions_grouped.file_date)
evictions_cleaned.file_date

0       2000-02-11
1       2000-09-19
2       1998-01-02
3       1998-01-02
4       1998-01-02
           ...    
43387   1997-12-31
43388   1997-12-31
43389   1997-12-31
43390   2000-09-19
43391   2000-02-14
Name: file_date, Length: 43392, dtype: datetime64[ns]

In [22]:
#convert constraints_date strings to datetime
evictions_cleaned.constraints_date = pd.to_datetime(evictions_grouped.constraints_date, errors = 'coerce')
evictions_cleaned.constraints_date

0       NaT
1       NaT
2       NaT
3       NaT
4       NaT
         ..
43387   NaT
43388   NaT
43389   NaT
43390   NaT
43391   NaT
Name: constraints_date, Length: 43392, dtype: datetime64[ns]

## Push to DB

In [37]:
#test row
evictions_test = evictions_cleaned.iloc[[1]]
evictions_test

,eviction_id,file_date,non_payment,breach,nuisance,illegal_use,failure_to_sign_renewal,access_denial,unapproved_subtenant,owner_move_in,...,substantial_rehab,ellis_act_withdrawal,condo_conversion,roommate_same_unit,other_cause,late_payments,lead_remediation,development,good_samaritan_ends,constraints_date
1,E2K2588,2000-09-19,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,NaT


In [38]:
#insert test row into eviction notices normalized table
evictions_test.to_sql("Rel_Eviction_Notices", engine, schema="dbo", if_exists="append", index=False)

2022-05-25 17:39:05,624 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2022-05-25 17:39:05,626 INFO sqlalchemy.engine.Engine [generated in 0.03766s] ('BASE TABLE', 'Rel_Eviction_Notices', 'dbo')
2022-05-25 17:39:05,865 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-25 17:39:05,923 INFO sqlalchemy.engine.Engine INSERT INTO dbo.[Rel_Eviction_Notices] (eviction_id, file_date, non_payment, breach, nuisance, illegal_use, failure_to_sign_renewal, access_denial, unapproved_subtenant, owner_move_in, demolition, capital_improvement, substantial_rehab, ellis_act_withdrawal, condo_conversion, roommate_same_unit, other_cause, late_payments, lead_remediation, development, good_samaritan_ends, constra

In [51]:
#check row was inserted correctly
statement = """
SELECT *
FROM Rel_Eviction_Notices;
"""

response = pd.read_sql_query(statement, connection)
response

2022-05-25 17:50:42,563 INFO sqlalchemy.engine.Engine 
SELECT *
FROM Rel_Eviction_Notices;

2022-05-25 17:50:42,575 INFO sqlalchemy.engine.Engine [raw sql] ()


,eviction_auto_id,eviction_id,file_date,non_payment,breach,nuisance,illegal_use,failure_to_sign_renewal,access_denial,unapproved_subtenant,...,ellis_act_withdrawal,condo_conversion,roommate_same_unit,other_cause,late_payments,lead_remediation,development,good_samaritan_ends,constraints_date,location_id
0,491FFCE4-4CED-4BF4-86D2-447A9E6A7B9E,E2K2588,2000-09-19,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,None,None
1,41891A8E-690C-4AE6-9B3A-66CAC91A25AB,E2K2588,2000-09-19,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,None,None
2,C7603E68-7EB6-46FB-B9EB-84299A62DBCD,E2K2588,2000-09-19,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,None,None
3,A76DA0CA-7BEE-467A-9FDC-B74DC894FFDA,E2K2588,2000-09-19,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,None,None


In [52]:
#drop test row inserted
statement = """
DELETE FROM Rel_Eviction_Notices
"""

connection.execute(statement)

2022-05-25 17:50:59,529 INFO sqlalchemy.engine.Engine 
DELETE FROM Rel_Eviction_Notices

2022-05-25 17:50:59,536 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-25 17:50:59,619 INFO sqlalchemy.engine.Engine COMMIT


In [53]:
#check row was dropped correctly
statement = """
SELECT *
FROM Rel_Eviction_Notices;
"""

response = pd.read_sql_query(statement, connection)
response

2022-05-25 17:51:16,277 INFO sqlalchemy.engine.Engine 
SELECT *
FROM Rel_Eviction_Notices;

2022-05-25 17:51:16,279 INFO sqlalchemy.engine.Engine [raw sql] ()


,eviction_auto_id,eviction_id,file_date,non_payment,breach,nuisance,illegal_use,failure_to_sign_renewal,access_denial,unapproved_subtenant,...,ellis_act_withdrawal,condo_conversion,roommate_same_unit,other_cause,late_payments,lead_remediation,development,good_samaritan_ends,constraints_date,location_id


In [60]:
#insert data for real
evictions_cleaned.to_sql("Rel_Eviction_Notices", engine, if_exists='append', index=False, chunksize=2000)

2022-05-25 17:57:14,210 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2022-05-25 17:57:14,211 INFO sqlalchemy.engine.Engine [generated in 0.00245s] ('BASE TABLE', 'Rel_Eviction_Notices', 'dbo')
2022-05-25 17:57:14,403 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-25 17:57:14,541 INFO sqlalchemy.engine.Engine INSERT INTO [Rel_Eviction_Notices] (eviction_id, file_date, non_payment, breach, nuisance, illegal_use, failure_to_sign_renewal, access_denial, unapproved_subtenant, owner_move_in, demolition, capital_improvement, substantial_rehab, ellis_act_withdrawal, condo_conversion, roommate_same_unit, other_cause, late_payments, lead_remediation, development, good_samaritan_ends, constraints

2022-05-25 18:01:21,624 INFO sqlalchemy.engine.Engine INSERT INTO [Rel_Eviction_Notices] (eviction_id, file_date, non_payment, breach, nuisance, illegal_use, failure_to_sign_renewal, access_denial, unapproved_subtenant, owner_move_in, demolition, capital_improvement, substantial_rehab, ellis_act_withdrawal, condo_conversion, roommate_same_unit, other_cause, late_payments, lead_remediation, development, good_samaritan_ends, constraints_date) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2022-05-25 18:01:21,631 INFO sqlalchemy.engine.Engine [cached since 247.2s ago] (('M010490', datetime.datetime(2001, 3, 16, 0, 0), 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'True', 'False', 'False', 'False', 'False', 'False', 'False', 'False', None), ('M010491', datetime.datetime(2001, 3, 16, 0, 0), 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'True', 'False', 'False', 'False', 'False', 'False'

2022-05-25 18:05:25,981 INFO sqlalchemy.engine.Engine INSERT INTO [Rel_Eviction_Notices] (eviction_id, file_date, non_payment, breach, nuisance, illegal_use, failure_to_sign_renewal, access_denial, unapproved_subtenant, owner_move_in, demolition, capital_improvement, substantial_rehab, ellis_act_withdrawal, condo_conversion, roommate_same_unit, other_cause, late_payments, lead_remediation, development, good_samaritan_ends, constraints_date) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2022-05-25 18:05:25,982 INFO sqlalchemy.engine.Engine [cached since 491.5s ago] (('M041790', datetime.datetime(2004, 10, 1, 0, 0), 'False', 'True', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', None), ('M041791', datetime.datetime(2004, 10, 1, 0, 0), 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False

2022-05-25 18:09:38,679 INFO sqlalchemy.engine.Engine INSERT INTO [Rel_Eviction_Notices] (eviction_id, file_date, non_payment, breach, nuisance, illegal_use, failure_to_sign_renewal, access_denial, unapproved_subtenant, owner_move_in, demolition, capital_improvement, substantial_rehab, ellis_act_withdrawal, condo_conversion, roommate_same_unit, other_cause, late_payments, lead_remediation, development, good_samaritan_ends, constraints_date) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2022-05-25 18:09:38,708 INFO sqlalchemy.engine.Engine [cached since 744.3s ago] (('M081725', datetime.datetime(2008, 10, 31, 0, 0), 'False', 'False', 'True', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', None), ('M081727', datetime.datetime(2008, 11, 7, 0, 0), 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'True', 'False', 'False

2022-05-25 18:13:45,106 INFO sqlalchemy.engine.Engine INSERT INTO [Rel_Eviction_Notices] (eviction_id, file_date, non_payment, breach, nuisance, illegal_use, failure_to_sign_renewal, access_denial, unapproved_subtenant, owner_move_in, demolition, capital_improvement, substantial_rehab, ellis_act_withdrawal, condo_conversion, roommate_same_unit, other_cause, late_payments, lead_remediation, development, good_samaritan_ends, constraints_date) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2022-05-25 18:13:45,107 INFO sqlalchemy.engine.Engine [cached since 990.7s ago] (('M130514', datetime.datetime(2013, 3, 22, 0, 0), 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'True', 'False', 'False', 'False', 'False', None), ('M130515', datetime.datetime(2013, 3, 22, 0, 0), 'False', 'False', 'True', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False'

2022-05-25 18:17:47,017 INFO sqlalchemy.engine.Engine INSERT INTO [Rel_Eviction_Notices] (eviction_id, file_date, non_payment, breach, nuisance, illegal_use, failure_to_sign_renewal, access_denial, unapproved_subtenant, owner_move_in, demolition, capital_improvement, substantial_rehab, ellis_act_withdrawal, condo_conversion, roommate_same_unit, other_cause, late_payments, lead_remediation, development, good_samaritan_ends, constraints_date) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2022-05-25 18:17:47,020 INFO sqlalchemy.engine.Engine [cached since 1233s ago] (('M160688', datetime.datetime(2016, 2, 24, 0, 0), 'False', 'False', 'False', 'True', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', None), ('M160689', datetime.datetime(2016, 2, 24, 0, 0), 'False', 'True', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False',

2022-05-25 18:21:52,823 INFO sqlalchemy.engine.Engine INSERT INTO [Rel_Eviction_Notices] (eviction_id, file_date, non_payment, breach, nuisance, illegal_use, failure_to_sign_renewal, access_denial, unapproved_subtenant, owner_move_in, demolition, capital_improvement, substantial_rehab, ellis_act_withdrawal, condo_conversion, roommate_same_unit, other_cause, late_payments, lead_remediation, development, good_samaritan_ends, constraints_date) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2022-05-25 18:21:52,824 INFO sqlalchemy.engine.Engine [cached since 1478s ago] (('M192412', datetime.datetime(2019, 9, 26, 0, 0), 'False', 'True', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', None), ('M192413', datetime.datetime(2019, 9, 26, 0, 0), 'False', 'False', 'False', 'False', 'False', 'True', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False',

2022-05-25 18:25:59,909 INFO sqlalchemy.engine.Engine INSERT INTO [Rel_Eviction_Notices] (eviction_id, file_date, non_payment, breach, nuisance, illegal_use, failure_to_sign_renewal, access_denial, unapproved_subtenant, owner_move_in, demolition, capital_improvement, substantial_rehab, ellis_act_withdrawal, condo_conversion, roommate_same_unit, other_cause, late_payments, lead_remediation, development, good_samaritan_ends, constraints_date) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2022-05-25 18:25:59,946 INFO sqlalchemy.engine.Engine [cached since 1726s ago] (('RE02130', datetime.datetime(1997, 6, 20, 0, 0), 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'True', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', None), ('RE02131', datetime.datetime(1997, 6, 20, 0, 0), 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False'

In [61]:
#check rows were inserted correctly
statement = """
SELECT *
FROM dbo.Rel_Eviction_Notices;
"""

response = pd.read_sql_query(statement, connection)
response

2022-05-25 18:27:19,970 INFO sqlalchemy.engine.Engine 
SELECT *
FROM dbo.Rel_Eviction_Notices;

2022-05-25 18:27:19,971 INFO sqlalchemy.engine.Engine [raw sql] ()


,eviction_auto_id,eviction_id,file_date,non_payment,breach,nuisance,illegal_use,failure_to_sign_renewal,access_denial,unapproved_subtenant,...,ellis_act_withdrawal,condo_conversion,roommate_same_unit,other_cause,late_payments,lead_remediation,development,good_samaritan_ends,constraints_date,location_id
0,E62C9649-A73C-498A-B032-000279D3A980,M2K0206,2000-02-09,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,None,None
1,470B6A02-2FD0-495A-AAD9-00050DD091B6,M163367,2016-10-24,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,None,None
2,51A18C15-A9EC-46C5-A9D5-0005601F5599,M2K1808,2000-09-08,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,None,None
3,C1EF514F-032B-430C-BEC2-00073B48763A,M040303,2004-02-24,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,None,None
4,5EBE4138-F21A-43F5-9E63-0008D9A9FB35,M080635,2008-04-08,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43387,790F468F-DE34-4FB3-BF60-FFF6B15DAB39,M172891,2017-11-27,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,None,None
43388,EC090B98-DD4C-4C9B-9A72-FFF6CB1D5844,M2K2283,2000-11-13,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,None,None
43389,1B7D9E27-E44D-4CCF-A6AD-FFFBF7273201,RE01766,1997-05-01,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,None,None
43390,DFDF7A61-2B92-47F4-A003-FFFD4639F917,M181946,2018-06-29,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,2023-07-30,None


In [93]:
#Possible TODO:
#for dates, once converted, check for outlier dates
#check to make sure dates make sense -- lots submitted on same date?